### 0 Helper function

In [72]:
import yaml

In [73]:
def load_file(filepath):
    with open(filepath,'r',encoding="utf-8") as f:
        return f.read()
def load_yaml(filepath):
    with open(filepath,'r',encoding="utf-8") as f:
        return yaml.safe_load(f)

<hr>

### 01 Input CVResume

In [74]:
resume_json = load_file("resume_json.txt")
print(resume_json)

{
  "contact_information": {
    "name": "Surya Teja Menta",
    "email": "-",
    "phone": "+91 8309584461",
    "linkedin": "-",
    "jobdb_link": "-",
    "portfolio_link": "suryatejamenta.co.in"
  },
  "professional_summary": {
    "has_summary": "Yes",
    "summary_points": [
      "I’m Surya Teja Menta, Results-driven Senior Data Scientist with 4+ years of experience in Data Science, Machine Learning (ML), and Generative AI (GenAI).",
      "Proven expertise in RAG (Retrieval-Augmented Generation), LLM fine-tuning, MLOps, and end-to-end AI solutions.",
      "Strong background in data analytics, statistical modeling, AI-powered automation, and scalable AI architectures.",
      "IBM Certified Professional Data Scientist with hands-on experience in LangChain, Hugging Face, OpenAI APIs, Vector Databases (ChromaDB, Pinecone), and cloud deployments (AWS, GCP).",
      "Passionate about AI research, model optimization, and developing cutting-edge AI solutions."
    ]
  },
  "education

<hr>

### 02 Promptbuilder

In [75]:
from google import genai
import json
import os
import yaml

In [76]:
class PromptBuilder:
    def __init__(self,section,criteria,cvresume):
        self.section  = section
        self.criteria = criteria[::-1]
        self.cvresume = cvresume
        with open("prompt.yaml","r",encoding="utf-8") as f:
            self.config = yaml.safe_load(f)
        # print(self.config)
    def build_response_template(self):
        return {
            "section": self.section,
            "scores": {
                c: {"score": 0, "feedback": ""} for c in self.criteria
            }
        }
    def build(self):
        config_role      = self.config['role']['role1']
        config_objective = self.config['objective']['objective1']
        config_section   = self.config['section']['section1']
        config_expected  = self.config['expected_content'][self.section]
        config_criteria  = ""
        for item in self.criteria:
            config_criteria = f"- {item}\n" + config_criteria
        config_scale     = self.config['scale']['score1']
        # config_output    = self.config['output']['format1']
        
        prompt_role      = f"Role :\n{config_role}"+"\n\n"
        prompt_objective = f"objectvie :\n{config_objective}"+"\n\n"
        prompt_section   = f"section :\n{config_section}"+"\n\n"
        prompt_expected  = f"expected :\n{config_expected}"+"\n"
        prompt_criteria  = f"Criteria :\n{config_criteria}"+"\n"
        prompt_scale     = f"Scale :\n{config_scale}"+"\n"
        prompt_output    = f"output :\n{json.dumps(self.build_response_template(), indent=2)}"+"\n\n"
        prompt_cvresume  = f"CV/Resume: \n{self.cvresume}"+"\n"

        prompt = prompt_role + prompt_objective + prompt_section \
            + prompt_expected + prompt_criteria + prompt_scale \
            + prompt_output + prompt_cvresume
        prompt = prompt.replace("<section_name>",self.section)
        return prompt

In [77]:
p = PromptBuilder( 
    section  = "Experience", 
    criteria = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    cvresume = resume_json
)
prompt = p.build()
print(prompt)

Role :
You are the expert HR evaluator

objectvie :
Evaluate the Experience section from the resume using the scoring criteria

section :
You are evaluating the Experience section.

expected :
- Job title, employer, dates
- Clear bullet points
- Action → method → impact structure
- Technical tools used
- Quantifiable metrics

Criteria :
- Completeness
- ContentQuality
- Grammar
- Length
- RoleRelevance

Scale :
0 = missing
1 = poor
2 = weak
3 = sufficient
4 = strong
5 = excellent

output :
{
  "section": "Experience",
  "scores": {
    "RoleRelevance": {
      "score": 0,
      "feedback": ""
    },
    "Length": {
      "score": 0,
      "feedback": ""
    },
    "Grammar": {
      "score": 0,
      "feedback": ""
    },
    "ContentQuality": {
      "score": 0,
      "feedback": ""
    },
    "Completeness": {
      "score": 0,
      "feedback": ""
    }
  }
}

CV/Resume: 
{
  "contact_information": {
    "name": "Surya Teja Menta",
    "email": "-",
    "phone": "+91 8309584461",
  

<hr>

### 03 LLMCaller

In [78]:
import re
import json

In [ ]:
class LlmCaller:
    def __init__(self,prompt):
        self.client = genai.Client(api_key="AIzaSy...")
        self.prompt = prompt
    def parser(self,resp):
        text = resp.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        data = json.loads(text)
        return data
    def hit(self):
        self.resp = self.client.models.generate_content(
            model    = "gemini-2.5-flash",
            contents = self.prompt
            )
        data = self.parser(self.resp)
        return data

In [80]:
x = LlmCaller(prompt)
op = x.hit()
op

{'section': 'Experience',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': 'The roles listed are directly aligned with data science, machine learning, and data analytics, showcasing a clear and consistent career path. The responsibilities described are highly relevant to a Senior Data Scientist profile.'},
  'Length': {'score': 5,
   'feedback': 'The descriptions for each role are appropriate, providing sufficient detail through concise bullet points without being overly wordy. Each role has a good number of impactful statements.'},
  'Grammar': {'score': 5,
   'feedback': 'The grammar, spelling, and punctuation throughout the experience section are flawless, demonstrating strong written communication skills.'},
  'ContentQuality': {'score': 3,
   'feedback': "While the section effectively highlights technical actions and tools used (e.g., YOLOv8, RAG pipelines, NLP Transformers), it significantly lacks quantifiable metrics to demonstrate the impact and scale of the achievement

<hr>

### 04 SectionScoreAggregator

In [81]:
import copy
class SectionScoreAggregator:
    def __init__(self,llm_output:dict):
        self.llm_output      = llm_output             # op
        self.config          = load_yaml("weight.yaml")         # config
        self.section         = llm_output["section"]  # Get section
        self.section_weights = self.config["weights"][self.section]  # config["weights"][section_key][criteria]
    def aggregate(self):
        ddict = {}
        total = 0.0
        # Protect multiple mutation when we run more than one time
        scores_copy = copy.deepcopy(self.llm_output["scores"])
        for criteria, body in scores_copy.items():
            raw = body["score"]
            w   = self.section_weights[criteria]
            weighted = raw / 5 * w
            body["score"] = weighted
            ddict[criteria] = body
            total = total + weighted
        return {
            "section": self.section,
            "total_score":total,
            "scores":ddict
        }

In [82]:
op

{'section': 'Experience',
 'scores': {'RoleRelevance': {'score': 5,
   'feedback': 'The roles listed are directly aligned with data science, machine learning, and data analytics, showcasing a clear and consistent career path. The responsibilities described are highly relevant to a Senior Data Scientist profile.'},
  'Length': {'score': 5,
   'feedback': 'The descriptions for each role are appropriate, providing sufficient detail through concise bullet points without being overly wordy. Each role has a good number of impactful statements.'},
  'Grammar': {'score': 5,
   'feedback': 'The grammar, spelling, and punctuation throughout the experience section are flawless, demonstrating strong written communication skills.'},
  'ContentQuality': {'score': 3,
   'feedback': "While the section effectively highlights technical actions and tools used (e.g., YOLOv8, RAG pipelines, NLP Transformers), it significantly lacks quantifiable metrics to demonstrate the impact and scale of the achievement

In [83]:
agg = SectionScoreAggregator(op)
s1 = agg.aggregate()
s1

{'section': 'Experience',
 'total_score': 42.0,
 'scores': {'RoleRelevance': {'score': 10.0,
   'feedback': 'The roles listed are directly aligned with data science, machine learning, and data analytics, showcasing a clear and consistent career path. The responsibilities described are highly relevant to a Senior Data Scientist profile.'},
  'Length': {'score': 10.0,
   'feedback': 'The descriptions for each role are appropriate, providing sufficient detail through concise bullet points without being overly wordy. Each role has a good number of impactful statements.'},
  'Grammar': {'score': 10.0,
   'feedback': 'The grammar, spelling, and punctuation throughout the experience section are flawless, demonstrating strong written communication skills.'},
  'ContentQuality': {'score': 6.0,
   'feedback': "While the section effectively highlights technical actions and tools used (e.g., YOLOv8, RAG pipelines, NLP Transformers), it significantly lacks quantifiable metrics to demonstrate the im

<hr>

### 05 GlobalAggregator

In [84]:
from datetime import datetime,timezone,timedelta

In [85]:
class GlobalAggregator:
    def __init__(self,SectionScoreAggregator_output:list):
        self.section_outputs = SectionScoreAggregator_output
        self.timestamp       = str(datetime.now(tz=(timezone(timedelta(hours=7)))))
        self.model_config    = load_yaml("model.yaml")     # should include model name
        self.weight_config   = load_yaml("weight.yaml")    # includes weights + version
        self.prompt_config   = load_yaml("prompt.yaml")    # includes prompt version
    def fn1(self):
        weights = self.weight_config["weights"]
        contribution = {}
        total = 0.0
        for section_data in self.section_outputs:
            section_name    = section_data["section"]
            total_score     = section_data["total_score"]
            section_weight  = weights[section_name]["section_weight"]
            section_contrib = total_score * section_weight
            contribution[section_name] = {
                "section_total": total_score,
                "section_weight": section_weight,
                "contribution": section_contrib
            }
            total = total + section_contrib
        return {
            "final_resume_score":total,
            "section_contribution":contribution
        }
    def fn2(self):
        details = {}
        for section_data in self.section_outputs:
            details[section_data["section"]] = {
                'total_score':section_data['total_score'],
                'scores':section_data['scores']
            }
        return details
    def fn3(self):
        return {
            "model_name": self.model_config['model']['generation_model'],
            "timestamp": self.timestamp,
            "weights_version": self.weight_config.get("version", "unknown"),
            "prompt_version": self.prompt_config.get("version", "unknown")
        }
    def fn0(self):
        conclution_part = self.fn1()
        detail_part     = self.fn2()
        metadata_part   = self.fn3()
        return {
            "conclution":conclution_part,
            "section_detail":detail_part,
            "metadata":metadata_part
        }

In [86]:
x = GlobalAggregator(
    SectionScoreAggregator_output = [s1]
)

In [87]:
x.fn0()

{'conclution': {'final_resume_score': 8.4,
  'section_contribution': {'Experience': {'section_total': 42.0,
    'section_weight': 0.2,
    'contribution': 8.4}}},
 'section_detail': {'Experience': {'total_score': 42.0,
   'scores': {'RoleRelevance': {'score': 10.0,
     'feedback': 'The roles listed are directly aligned with data science, machine learning, and data analytics, showcasing a clear and consistent career path. The responsibilities described are highly relevant to a Senior Data Scientist profile.'},
    'Length': {'score': 10.0,
     'feedback': 'The descriptions for each role are appropriate, providing sufficient detail through concise bullet points without being overly wordy. Each role has a good number of impactful statements.'},
    'Grammar': {'score': 10.0,
     'feedback': 'The grammar, spelling, and punctuation throughout the experience section are flawless, demonstrating strong written communication skills.'},
    'ContentQuality': {'score': 6.0,
     'feedback': "W